Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cobra
import escher

# Load model

In [ ]:
model = cobra.io.load_json_model("./models/iMM904.json")

# Utilities

In [ ]:
def print_formulas(reaction):
    """Print formulas of reactants and products of a reaction."""
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id} ({reactant.name}): F {reactant.formula}')
    print('products')
    for product in reaction.products:
        print(f'{product.id} ({product.name}): F {product.formula}')

def print_formula_weights(reaction):
    """Print formula weights of reactants and products of a reaction."""
    print('reactants')
    for reactant in reaction.reactants:
        print(f'{reactant.id} ({reactant.name}): MW {reactant.formula_weight}')
    print('products')
    for product in reaction.products:
        print(f'{product.id} ({product.name}): MW {product.formula_weight}')
        
def print_stoichiometry(reaction):
    """Pretty-print stoichiometry of reaction."""
    for metabolite, coeff in reaction.metabolites.items():
        print(f'{coeff}, {metabolite.name}')

# Objective function

Get objective function (biomass/growth)

In [ ]:
biomass = model.reactions.BIOMASS_SC5_notrace
biomass

In [ ]:
print_stoichiometry(biomass)

> These stoichiometric constants were determined by the relative compositions (i.e. mass fractions) of these compounds in exponentially growing yeast, as determined from experiments (Mo et al., 2009).

Medium

In [ ]:
model.medium

Unrestrict bounds

In [ ]:
model.reactions.get_by_id('EX_glc__D_e').bounds = (-10, 0)
model.reactions.get_by_id('EX_o2_e').bounds = (-999999.0, 0)

> Problem: If I set the bounds of glucose exchange to be (-99999, 0), the objective function reaches a huge value, so I left it at the default value.

Linear reaction coefficients

In [ ]:
cobra.util.solver.linear_reaction_coefficients(model)

> The objective function is the biomass reaction and the biomass reaction only, not a linear combination of anything.

Optimise using (vanilla) FBA

In [ ]:
solution = model.optimize()

In [ ]:
model.summary()

In [ ]:
solution

# Fluxes

Import list of metabolites represented in biomass reaction.  `biomass_type` is manually labelled.

In [ ]:
biomass_metabolites_df = pd.read_csv('iMM904-biomass-categories.csv', delimiter=',')

In [ ]:
biomass_metabolites_df

Get fluxes of each metabolite in the (optimised) biomass reaction, append to the dataframe.

In [ ]:
model.metabolites.get_by_id('so4_c').summary()

> There is one producing reaction so I take the flux of this metabolite through this reaction (-1.106).  For other reactions, I take the sum.  Below, I repeat this process for all metabolites represented in the reactants of the biomass reaction.

In [ ]:
biomass_metabolites_df['flux'] = [np.nan] * len(biomass_metabolites_df)

In [ ]:
# ignore this
for df_idx, metabolite_id in enumerate(biomass_metabolites_df.id):
    metabolite = model.metabolites.get_by_id(metabolite_id)
    # Inspect reactants, ignore products, based on stoichiometric constants
    if biomass.metabolites[metabolite] < 0:
        flux_in_biomass = metabolite.summary().consuming_flux.loc['BIOMASS_SC5_notrace'].flux
        #print(f'{metabolite.name}: {-1/flux_in_biomass}')
        biomass_metabolites_df['flux'].iloc[df_idx] = flux_in_biomass

In [ ]:
# alternatively
for df_idx, metabolite_id in enumerate(biomass_metabolites_df.id):
    metabolite = model.metabolites.get_by_id(metabolite_id)
    # Inspect reactants, ignore products, based on stoichiometric constants
    if biomass.metabolites[metabolite] < 0:
        flux_producing = sum(metabolite.summary().producing_flux.flux)
        #print(f'{metabolite.name}: {-1/flux_in_biomass}')
        biomass_metabolites_df['flux'].iloc[df_idx] = flux_producing

In [ ]:
biomass_metabolites_df

# Timescale

## From objective function

The objective function is expressed in units of mmol gDW-1 h-1

In [ ]:
solution.objective_value

Converting this into estimated time:

In [ ]:
CELL_DRY_MASS = 15e-12

#time = CELL_DRY_MASS/solution.objective_value
time = 1/solution.objective_value

In [ ]:
time

## From fluxes

Compute sum of reactant coefficients.

This is needed to calculate the mass fraction of each metabolite.  This calculation is based on the assumption that the relative values of the reactant coefficients reflect the mass fraction of each metabolite, as suggested by Mo et al. (2009).

In [ ]:
sum_reactant_coeffs = -sum([coeff for (_, coeff) in biomass.metabolites.items() if coeff < 0])
sum_reactant_coeffs

Remove the coeffs from NGAM as these ones were considered separately when constructing the biomass reaction of the model.

In [ ]:
sum_reactant_coeffs -= -biomass.metabolites[model.metabolites.get_by_id('atp_c')]
sum_reactant_coeffs -= -biomass.metabolites[model.metabolites.get_by_id('h2o_c')]
sum_reactant_coeffs

Construct new columns in `biomass_metabolites_df`.

In [ ]:
biomass_metabolites_df['stoich'] = [np.nan] * len(biomass_metabolites_df)
biomass_metabolites_df['mass_fraction'] = [np.nan] * len(biomass_metabolites_df)
biomass_metabolites_df['formula_weight'] = [np.nan] * len(biomass_metabolites_df)

for df_idx, metabolite_id in enumerate(biomass_metabolites_df.id):
    metabolite = model.metabolites.get_by_id(metabolite_id)
    biomass_metabolites_df['stoich'].iloc[df_idx] = biomass.metabolites[metabolite]
    # Inspect reactants, ignore products, based on stoichiometric constants
    if biomass.metabolites[metabolite] < 0:
        if metabolite_id in ['atp_c', 'h2o_c']:
            biomass_metabolites_df['mass_fraction'].iloc[df_idx] = np.nan
        else:
            biomass_metabolites_df['mass_fraction'].iloc[df_idx] = -biomass.metabolites[metabolite]/sum_reactant_coeffs
        biomass_metabolites_df['formula_weight'].iloc[df_idx] = metabolite.formula_weight

Estimate time.

Logic:
- For metabolite $s_i$, the mass of $s_i$ that must be produced is equal to (mass of cell) $\times$ (coeff of $s_i$)/(sum of coeffs).  (coeff of $s_i$)/(sum of coeffs) can alternatively be expressed as the mass fraction of $s_i$.
- The number of moles that this mass represents is thus equal to (mass $s_i$)/(MW $s_i$).
- The time is thus equal to (mol $s_i$)/((flux in mmol/gDW/h)(mass of cell)).
- This works out to (1000 $\times$ mass fraction $s_i$)/((MW $s_i$)(flux in mmol/gDW/h)).  The 1000 is there because molecular weight is expressed as g/mol while flux is in mmol/gDW/h.

In [ ]:
biomass_metabolites_df['time'] = -1000 * biomass_metabolites_df['mass_fraction'] / (biomass_metabolites_df['formula_weight'] * biomass_metabolites_df['flux'])

In [ ]:
biomass_metabolites_df

In [ ]:
biomass_metabolites_df['time'].sum()